<div style="background-color: #CD5C5C; padding: 20px; border-radius: 10px; box-shadow: 5px 5px 10px #888888;">
  <h1 style="color: white; font-size: 30px; font-weight: bold; text-align: center; text-shadow: 2px 2px 4px #000000;">Sports Data Analysis: Problem Statement 3
</h1>
</div>

## Data Source  

The data source for this project is in the form of csv and understanding the nature, format is essential as it impacts the stages of ingestion and transformation.

## 1. Connecting to MySQL Database  

Establishing a communication channel between our Python program and the database management system(MySQL). It is a crucial step in the data ingestion process, bringing data from the database into the Python environment for further analysis.  

In [1]:
#Connecting to mysql workbench database
import pandas as pd
import mysql.connector
from mysql.connector import Error
import logging

#Database connection parameters
db_config = {
    'host': 'localhost',
   # 'database': 'sports_data',
    'user': 'root',
    'password': 'HithaChoudhary.21',
    'port': '3307'
}

#Function to connect to the MySQL database
def connect_to_database():
    try:
        connection = mysql.connector.connect(**db_config)
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print(f"Error connecting to MySQL database: {e}")

connect_to_database()

C:\Users\hithachoudhary\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\hithachoudhary\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


Connected to MySQL database


## 2. Data Ingestion Strategy  

### 2.1 Setting up the table and database to further ingest the data from the csv file in a batch  
 
-->The below code also includes logging to track the performance and reliability of the ingestion process. The 'data_ingestion.log' file is created to store the tracking process.  
-->This stage also includes creation of index for improved query performance and faster data retrieval.  

In [13]:
import concurrent.futures
import time

#Configuring logging
logging.basicConfig(level=logging.INFO, filename='data_ingestion.log', filemode='a', format='%(asctime)s - %(levelname)s - %(message)s')

db_config = {
    'host': 'localhost',
    'user': 'root',
    'database': 'sports_data',
    'password': 'HithaChoudhary.21',
    'port': '3307'
}

conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()
    
cursor.execute('''
        CREATE TABLE IF NOT EXISTS football_data (
            ID INT PRIMARY KEY,
            Unnamed INT,
            Player VARCHAR(60),
            Team VARCHAR(60),
            Age INT,
            Height FLOAT,
            Weight FLOAT,
            Position VARCHAR(50),
            Goals INT,
            Assists FLOAT,
            YellowCards INT,
            RedCards INT,
            PassCompletionRate FLOAT,
            DistanceCovered FLOAT,
            Sprints INT,
            ShotsOnTarget INT,
            TacklesWon INT,
            CleanSheets INT,
            PlayerFatigue FLOAT,
            MatchPressure INT,
            InjuryHistory INT,
            TrainingHours FLOAT,
            FatigueInjuryCorrelation FLOAT,
            PressurePerformanceImpact FLOAT,
            EffectiveTraining FLOAT,
            Season INT, 
            GoalsPerMatch FLOAT,
            AssistsPerMatch FLOAT
        )
    ''')
#Creation of index on the column positions for faster querying on that column.
cursor.execute("SHOW INDEX FROM football_data WHERE Key_name = 'idx_position'")
index_exists = cursor.fetchone()
if not index_exists:
    cursor.execute('CREATE INDEX idx_position ON football_data (Position)')
#Commiting the transaction
conn.commit()
conn.close()
print("Database and table setup complete.")

Database and table setup complete.


## 3. Data Processing Plan  

ETL(Extract,Transform,Load): This process includes extracting the data from the source, transforming the data with the help of python and loading it to the mysql databse server. The fusion of python and mysql helps complete the ETL process.

-->The third and final process of ETL; loading of the data to the MySQL database is achievable with this code.  
-->Incremental Loading is included in the code snippet below where it processess only new or modified data since the last successful data load.

In [18]:
def load_new_data(csv_file):
    logging.info('Loading new data from CSV file.')
    
    #Loading the new data from the CSV file
    new_data = pd.read_csv(csv_file)
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    #Get existing indices from the database
    cursor.execute("SELECT ID FROM football_data")
    existing_indexes = [row[0] for row in cursor.fetchall()]
    #Filtering out records that already exist in the database
    new_records = new_data[~new_data['ID'].isin(existing_indexes)]
    logging.info(f'Found {len(new_records)} new records to insert.')
    #Inserting new records into the database
    for _, row in new_records.iterrows():
        cursor.execute('''
            INSERT INTO football_data (
                'ID', 'Unnamed', 'Player', 'Team', 'Age', 'Height', 'Weight', 'Position',
                'Goals', 'Assists', 'YellowCards', 'RedCards', 'PassCompletionRate',
                'DistanceCovered', 'Sprints', 'ShotsOnTarget', 'TacklesWon',
                'CleanSheets', 'PlayerFatigue', 'MatchPressure', 'InjuryHistory',
                'TrainingHours', 'FatigueInjuryCorrelation', 'PressurePerformanceImpact',
                'EffectiveTraining', 'Season', 'GoalsPerMatch', 'AssistsPerMatch'
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', tuple(row))
    conn.commit()
    conn.close()
    
    logging.info(f'{len(new_records)} new records inserted.')

load_new_data('sports_dataset_new.csv')

#### The above created design for a data ingestion pipeline is able to meet the requirements of:  
(1)Connecting to the MySQL server database  
(2)Ingesting the data using batch processing   
(3)Creation of index to optimize storage  
(4)Implementation of logging to monitor and track the ingestion process